## 라이브러리 & API Key 정의

In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import re

load_dotenv() # .env 파일 로드
my_api_key = os.getenv("API_KEY") # 환경 변수에서 API 키 불러오기

client = OpenAI(
    api_key = my_api_key
)

## Instuction 정의

In [5]:
# negative용
instruction_1 = """
You are a recommendation bot responsible for selecting the news article that the target user is most likely to prefer from a list of five candidate articles. The only information available for each candidate article is its title, which is written in Norwegian.

Your goal is to predict the index number of the news article that best fits in the position labeled [MASK].
"""

instruction_negative = """
You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News is provided by title only.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news that the user is most likely to read.
"""

# category용
instruction_negative = """
You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news that the user is most likely to read.
"""

# category용 + ndcg
instruction_negative = """
You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
The answers should sort the index numbers of the news articles in the order that you think best fits the [MASK] based on the user's preferences (only the top 10 will be returned).
"""

# category용 + ndcg2
instruction_negative = """You are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered. """



instruction_positive = """
You are a bot that identifies users' news interests from [Click History], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News is provided by title only.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
The answer should return only one news that the user is most likely to read.
"""

# category용 + ndcg
instruction_positive = """
You are a bot that identifies users' news interests from [Click History], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
The answers should sort the index numbers of the news articles in the order that you think best fits the [MASK] based on the user's preferences (only the top 10 will be returned).
"""

# category용 + ndcg2
instruction_positive = """You are a bot that identifies users' news interests from [Click History], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].

News only provides a title and category.
News is Norwegian news in Norwegian.

[Questions] can have multiple questions, each of which must be answered. """

In [18]:
# 0217 ndcg
instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
"""


instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.

<Input example 1>
[Click History]
The news articles that User #15001 clicked before are as follows:
1. Gravid i femte måned - vant NM-finalen [category : ballsport]
2. Trafikkuhell i Vemundvik [category : nordtrondelag]
3. Norgesgruppen kjøper seg inn i grensehandelkjede [category : okonomi]
...

[Questions]
Based on User #15001's preferences, arrange the index numbers of the top five news articles in the sequence that is deemed most suitable for [MASK]

Question 1) User #15001 prefers [MASK] the most among the following five articles: 
1: Silver ber om midlertidig forbud mot offentlig administrasjon [category : okonomi]
2: Perfekt gripefølelse [category : digital]
3: Svenssons debut i Nederland utsettes: Ikke i kveldens Europa League-tropp [category : fotball]
4: Nye tall: Inntil 91 ulver i Norge [category : innenriks]
5: Trygg Trafikk vil ha alkolås i alle biler [category : innenriks]

<Output example 1> 
Question 1: 3, 1, 4, 5, 2

<Input example 2>
[Click History]
The news articles that User #15002 clicked before are as follows:
1. Nå åpner byens nye turløype [category : trondheim]
2. - Jeg har litt erfaring med steinhugging fra før [category : trondheim]
3. «Det mest ettertraktede området i vårt nærområde står i fare for å bli avstengt for allmennheten» [category : ordetfritt]
...

[Questions]
Based on User #15002's preferences, arrange the index numbers of the top five news articles in the sequence that is deemed most suitable for [MASK]

Question 1) User #15002 prefers [MASK] the most among the following five articles: 
1: eAdressa er oppdatert [category : okonomi]
2: Arkitekter reagerer på nytt boligforslag [category : innenriks]
3: Han er en av 13 som får si sitt om fremtidens hær og heimevern [category : nyheter]
4: Hun får halve verden til Trondheim [category : magasin]
5: Kjenner jeg ekstra godt etter så tror jeg at jeg føler meg litt sånn «hin» [category : ordetfritt]

<Output example 2> 
Question 1: 5, 2, 3, 1, 4

<Input example 3>
[Click History]
The news articles that User #15003 clicked before are as follows:
1. Nå åpner byens nye turløype [category : trondheim]
2. Orkangers keiserinne [category : sortrondelag]
3. Mathilde (3) måtte returnere til St. Olavs Hospital etter en dag på Værnes [category : nordtrondelag]
...

[Questions]
Based on User #15003's preferences, arrange the index numbers of the top five news articles in the sequence that is deemed most suitable for [MASK]

Question 1) User #15003 prefers [MASK] the most among the following five articles: 
1: 19-åring tiltalt for gjengvoldtekt i Tromsø [category : innenriks]
2: Innbrudd hos bedrift i Osloveien [category : trondheim]
3: Som vekter i tolv år har jeg fått et ubehagelig innsyn i hva unge spiser i langfriminuttet [category : ordetfritt]
4: Kvinne døde i påkjørsel på E18 ved Horten [category : innenriks]
5: Treneren etter den norske fiaskoen: - Det er nitrist [category : vintersport]

<Output example 3> 
Question 1: 3, 2, 1, 5, 4
"""

In [19]:
# 0204 acc
instruction_negative = """
You are a bot designed to identify users' news interests based on their [News of Interest to the user] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

There can be multiple lists in [News of Interest to the user], each with five news items.
Among the five news in each list, there is one news that the user is most interested in.

[Questions] can have multiple questions, each of which must be answered.
"""



instruction_positive = """
You are a bot designed to identify users' news interests based on their [Click History] and predict the index number of news items in [Questions] that best fit the position labeled [MASK].

Each news article contains only a title and category written in Norwegian.

[Questions] can have multiple questions, each of which must be answered.
"""

In [5]:
instruction_negative

"\nYou are a bot that identifies users' news interests from [News of Interest to the user], then based on this, predicts the index number of news in [Questions] that best fits in the position labeled [MASK].\n\nNews only provides a title and category.\nNews is Norwegian news in Norwegian.\n\nThere can be multiple lists in [News of Interest to the user], each with five news items.\nAmong the five news in each list, there is one news that the user is most interested in.\n\n[Questions] can have multiple questions, each of which must be answered.\nThe answer should return only one news that the user is most likely to read.\n"

## inference 함수 정의

In [20]:
def inference(purpose, target_folder, result_file_name, gpt_model, user_list, max_attempts):

    # instruction 정의
    if purpose == 'with_negative': 
        instruction = instruction_negative
    elif purpose == 'only_positive':
        instruction = instruction_positive

    # User Prompt가 위치한 폴더 및 metadata 파일 경로 설정
    target_folder = f'../../prompts/{target_folder}'
    directory = f'{target_folder}/{purpose}'
    meta_file_path = f'{target_folder}/{purpose}/metadata/output_metadata.txt'
    user_question_counts = {}
    
    # metadata 파일을 읽어 user별 question 수 저장
    with open(meta_file_path, 'r', encoding='utf-8') as meta_file:
        for line in meta_file:
            match = re.match(r'User ID:\s*U(\d+).*Question 수:\s*(\d+)', line)
            if match:
                user_id = int(match.group(1))
                question_count = int(match.group(2))
                user_question_counts[user_id] = question_count

    # 실험 실행
    with open(f'../../results/gpt_result/{result_file_name}', 'w', encoding='utf-8') as result_file:
        # user list에서 각 user에 대해 처리
        for cnt, i in enumerate(user_list):
            filename = f'U{i}.txt'
            filepath = os.path.join(directory, filename)
            
            # 파일 존재 여부 확인
            if os.path.isfile(filepath):
                # 파일 내용 읽기
                with open(filepath, 'r', encoding='utf-8') as f:
                    contents = f.read()

                # user의 question 수 설정
                expected_question_count = user_question_counts.get(i)
                if expected_question_count is None:
                    print(f"사용자 U{i}의 질문 수를 찾을 수 없습니다.")
                    continue  # 다음 사용자로 넘어감
                
                # API 요청 준비
                initial_messages = [
                    {"role": "system", "content": instruction},
                    {"role": "user", "content": contents}
                ]
                messages = initial_messages.copy()
                attempt = 0

                # 최대 시도 횟수를 넘지 않았으면 실행
                while attempt < max_attempts:
                    attempt += 1
                    # API 호출
                    try:
                        response = client.chat.completions.create(
                            model=gpt_model,
                            messages=messages
                        )
                    except Exception as e:
                        print(f"API 호출 중 오류 발생 (사용자 {i}): {e}")
                        break  # 다음 사용자로 넘어감
                    
                    # 응답 내용 추출
                    response_text = response.choices[0].message.content.strip()
                    
                    
                    result_file.write(f'[U{i}]\n')
                    result_file.write(response_text + '\n\n')
                    if (cnt+1) % 20 == 0:
                        print(f'☆ {purpose} U{i} 까지 완료 [{cnt+1}/{len(user_list)}] ☆')  
                    break  # 루프 종료
                    
            else:
                print(f'파일 {filepath} 이 존재하지 않습니다.')
        print(f'{purpose} 완료 : {result_file_name}\n')


In [27]:
user_range = 1000
users = [i for i in range(1, user_range + 1)]

# users =  [15168, 15473]

# 실행


# users = [1, 3, 10, 13, 14, 15, 19, 20, 26, 27, 31, 32, 36, 39, 41, 44, 47, 50, 51, 53, 55, 57, 61, 62, 73, 76, 80, 83, 88, 90, 96, 101, 102, 104, 106, 108, 110, 114, 117, 118, 119, 120, 122, 126, 127, 129, 132, 134, 135, 139, 141, 144, 149, 150, 152, 155, 156, 160, 162, 165, 168, 169, 170, 171, 174, 175, 177, 178, 181, 182, 183, 187, 195, 198, 201, 202, 205, 207, 208, 209, 211, 214, 220, 221, 223, 227, 228, 229, 230, 232, 235, 242, 243, 245, 249, 250, 251, 252, 253, 254, 258, 259, 260, 262, 266, 267, 268, 269, 271, 276, 278, 282, 286, 288, 297, 302, 304, 306, 310, 316, 317, 336, 344, 347, 351, 352, 353, 356, 358, 359, 362, 365, 368, 369, 374, 382, 384, 389, 395, 396, 401, 408, 410, 411, 412, 418, 421, 426, 428, 430, 434, 438, 440, 441, 445, 447, 451, 452, 454, 457, 459, 460, 462, 463, 464, 469, 475, 480, 481, 482, 486, 487, 490, 496, 498, 501, 503, 506, 508, 511, 518, 522, 524, 526, 527, 528, 532, 540, 542, 545, 546, 547, 549, 551, 552, 553, 560, 561, 562, 565, 572, 575, 577, 581, 584, 586, 587, 589, 591, 595, 599, 600, 605, 610, 611, 614, 615, 617, 620, 623, 624, 629, 630, 636, 642, 644, 653, 654, 657, 658, 659, 661, 663, 665, 666, 670, 672, 673, 676, 677, 678, 680, 682, 687, 689, 692, 693, 694, 695, 697, 700, 702, 703, 704, 706, 707, 708, 709, 710, 713, 717, 731, 736, 738, 743, 752, 754, 757, 759, 761, 774, 775, 776, 778, 779, 782, 785, 786, 788, 797, 798, 800, 807, 811, 813, 817, 818, 819, 820, 824, 831, 832, 841, 843, 845, 846, 847, 850, 855, 856, 859, 861, 862, 866, 867, 868, 875, 876, 881, 884, 887, 888, 889, 890, 891, 893, 896, 900, 901, 907, 908, 910, 922, 924, 926, 927, 928, 929, 931, 933, 938, 948, 949, 950, 953, 958, 963, 965, 967, 976, 981, 983, 984, 985, 987, 999, 1000]

users = [845]
inference(purpose='only_positive', 
          target_folder='[top1] test_ns4',
          result_file_name='[250310] positive_ns4_fine(40,15)_across(negative_model)2.txt',
          gpt_model='ft:gpt-4o-mini-2024-07-18:personal:across-users-ns4-40-15-negative-random:B9ROFM5c:ckpt-step-80', 
          user_list=users, 
          max_attempts=1
          )

users = [594]
inference(purpose='with_negative', 
          target_folder='[top1] test_ns4',
          result_file_name='[250310] negative_ns4_fine(40,15)_across(positive_model)2.txt',
          gpt_model='ft:gpt-4o-mini-2024-07-18:personal:across-users-ns4-40-15-positive-random:B9S14D5m:ckpt-step-80', 
          user_list=users, 
          max_attempts=1
          )

only_positive 완료 : [250310] positive_ns4_fine(40,15)_across(negative_model)2.txt

with_negative 완료 : [250310] negative_ns4_fine(40,15)_across(positive_model)2.txt



In [16]:
user_range = 1000
users = [i for i in range(1, user_range + 1)]

# 실행


inference(purpose='with_negative', 
          target_folder='[top1] test_ns4',
          result_file_name='[250308] negative_ns4_fine_across_posi_model.txt',
          gpt_model='ft:gpt-4o-mini-2024-07-18:personal:positive-ns4-train40-val15:B81XWTKR:ckpt-step-80', 
          user_list=users, 
          max_attempts=1
          )

☆ with_negative U20 까지 완료 [20/1000] ☆
☆ with_negative U40 까지 완료 [40/1000] ☆
☆ with_negative U60 까지 완료 [60/1000] ☆
☆ with_negative U80 까지 완료 [80/1000] ☆
☆ with_negative U100 까지 완료 [100/1000] ☆
☆ with_negative U120 까지 완료 [120/1000] ☆
☆ with_negative U140 까지 완료 [140/1000] ☆
☆ with_negative U160 까지 완료 [160/1000] ☆
☆ with_negative U180 까지 완료 [180/1000] ☆
☆ with_negative U200 까지 완료 [200/1000] ☆
☆ with_negative U220 까지 완료 [220/1000] ☆
☆ with_negative U240 까지 완료 [240/1000] ☆
☆ with_negative U260 까지 완료 [260/1000] ☆
☆ with_negative U280 까지 완료 [280/1000] ☆
☆ with_negative U300 까지 완료 [300/1000] ☆
☆ with_negative U320 까지 완료 [320/1000] ☆
☆ with_negative U340 까지 완료 [340/1000] ☆
☆ with_negative U360 까지 완료 [360/1000] ☆
☆ with_negative U380 까지 완료 [380/1000] ☆
☆ with_negative U400 까지 완료 [400/1000] ☆
☆ with_negative U420 까지 완료 [420/1000] ☆
☆ with_negative U440 까지 완료 [440/1000] ☆
☆ with_negative U460 까지 완료 [460/1000] ☆
☆ with_negative U480 까지 완료 [480/1000] ☆
☆ with_negative U500 까지 완료 [500/1000] ☆
☆ with_n